In [37]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import math
import scipy.stats
from IPython.display import HTML
import tqdm
%matplotlib inline

In [39]:
#扱う関数
def f(x, y):
    return x**2 + y**2

#勾配と偏微分
h = 1.0 * 10**-7
def grad_f(x,y):
    grad_x = (f(x + h, y) - f(x, y)) / h
    grad_y = (f(x, y + h) - f(x, y)) / h
    return [grad_x, grad_y]

def partial_grad_f(x,y,index):
    if index == 0:
        grad_x = (f(x + h, y) - f(x, y)) / h
        grad_y = 0
    else:
        grad_x = 0
        grad_y = (f(x, y + h) - f(x, y)) / h
    return [grad_x, grad_y]
    


In [22]:
#パラメータ更新
def optimize(x,y,step_num = 1000, learning_rate=1e-2,momentum=0.8, mode ="GD"):
    start = (x,y)
    points = []
    if mode == "GD":
        for i in range(step_num+1):
            x,y = start
            gradient = grad_f(x,y)
            x_new = x - learning_rate * gradient[0]
            y_new = y - learning_rate * gradient[1]
            points.append([x,y])
            start = (x_new,y_new)
    elif mode == "momentum":
        v_x, v_y = 0, 0 # the momentum point
        for i in range(step_num+1):
            x,y = start
            gradient = grad_f(x, y)
            v_x = momentum * v_x - learning_rate * gradient[0]
            v_y = momentum * v_y - learning_rate * gradient[1]
            x_new = x + v_x
            y_new = y + v_y
            points.append([x,y])
            start = (x_new,y_new)
    elif mode == "ACDM":
        gamma = 1/8.0
        c= 1/8.0
        v_x, v_y = 0, 0 # the momentum point
        for i in range(step_num+1):
            gamma_temp = gamma
            gamma = math.sqrt(c**2+gamma_temp**2) + c
            alpha = gamma/(4.0*gamma_temp**2+gamma)
            random = scipy.stats.bernoulli.rvs(0.5)
            x,y = start
            gradient = partial_grad_f(alpha*v_x+(1.0-alpha)*x, alpha*v_y+(1.0-alpha)*y, random)
            x_new = alpha*v_x+(1.0-alpha)*x - learning_rate * gradient[0]
            y_new = alpha*v_y+(1.0-alpha)*y - learning_rate * gradient[1]
            v_x = v_x - gamma*learning_rate * gradient[0]
            v_y = v_y - gamma*learning_rate * gradient[1]
            points.append([x,y])
            start = (x_new,y_new)
    else:
        print("Error")
    return points
        
        
            



    
    

In [43]:
#グラフ関係の設定
fig = plt.figure()
ax = fig.add_subplot(111, aspect=1)
x = np.linspace(-50, 50, 100)
y = np.linspace(-50, 50, 100).reshape(-1, 1)
#ハイパーパラメータ設定
hyperparameters={"x":50,
"y":50,
"step_num" : 100,
"learning_rate":1e-2,
"momentum":0.8,
"mode":"ACDM"}
points = optimize(**hyperparameters)
def update(z):
    ax.cla() # ax をクリア
    ax.pcolormesh(x,y,f(x,y))
    ax.plot(points[z][0],points[z][1], "o", c="red")

anim = FuncAnimation(fig, update, frames=np.arange(hyperparameters["step_num"]), interval=200)

#anim.save("c03.gif", writer="imagemagick")
plt.close()
HTML(anim.to_jshtml())